In [ ]:
#Import the necessary modules

In [ ]:
import os
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import pandas as pd

from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Locate the directory

In [ ]:
weight_path = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
BASE_DATA_FOLDER = "../input/plant-seedlings-classification/train"
num_classes = len(os.listdir(BASE_DATA_FOLDER))
image_size = 224

In [ ]:
#Prepare the netork Using VGG16 model and last densly connected layer

In [ ]:
my_model = Sequential()
my_model.add(ResNet50(include_top=False, pooling='avg', weights=weight_path))
my_model.add(Dense(num_classes, activation='softmax'))
my_model.layers[0].trainable = True
my_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Prepare the data generator and use data augmentation

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    horizontal_flip=True,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2)

train_generator = data_generator.flow_from_directory(directory='../input/plant-seedlings-classification/train',
                                                     target_size=(image_size,image_size),
                                                     class_mode='categorical')

In [ ]:
# Map the Lbale with index 
label = train_generator.class_indices
# Make a dictionary key as Label name and index as value ex: {'Black-grass:0}..
label = dict((v,k) for k,v in label.items())

# Train the generator
my_model.fit_generator(train_generator,epochs=5)

In [ ]:
# Predict and prepare the prediction for submission

In [ ]:
test_path = '../input/plant-seedlings-classification/test'
train_path = '../input/plant-seedlings-classification/train'
sample_path = '../input/plant-seedlings-classification/sample_submission.csv'

raw_sample = pd.read_csv(sample_path)
ImageId_sample = raw_sample.values[:,0]

label_map = {}
preds = []
l = 0
for label_name in ImageId_sample:
    imgPath = os.path.join(test_path,label_name)
    Image = load_img(imgPath, target_size=(image_size,image_size))
    Image = img_to_array(Image)
    Image = np.expand_dims(Image,axis=0)
    pred = my_model.predict(Image)
    pred = np.argmax(pred)
    pred = label[pred]
    preds.append(pred)

my_submission = pd.concat([pd.Series(ImageId_sample,name="file"), pd.Series(preds,name="species")], axis=1)
my_submission.to_csv('submission.csv', index=False)